In [5]:
# Imports

import json
import geojson
import geopandas
import pandas as pd
import pathlib
import csv
import copy
import ast
import os
import time
from collections import OrderedDict

from geojson import Feature, Point, FeatureCollection

In [6]:
source_data_path = '/Users/alanmccann/Dropbox/bain/bain-uploads/intermarketdata-20230111.json'
dest_data_path = '/Users/alanmccann/Dropbox/bain/bain-uploads/intermarketdata-20230111-flows.json'

In [7]:
def read_intramarket_json(file_path):
    with open(file_path) as f:
        data = json.load(f)
    print(f'Loaded data from {file_path}')
    return  data

def write_data_json(json_data, path):
    json_string = json.dumps(json_data)
    f = open(path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote intermarket json data to  {path}')



In [8]:
data = read_intramarket_json(source_data_path)

Loaded data from /Users/alanmccann/Dropbox/bain/bain-uploads/intermarketdata-20230111.json


In [9]:
data['states']['Texas']['Alaska']

[0.004,
 0.001,
 0.007,
 0.004,
 0.004,
 0.001,
 0.006,
 0.004,
 0.005,
 0.002,
 0.008,
 0.004,
 0.003,
 0.001,
 0.006,
 0.003,
 0.004,
 0.005,
 0.004,
 0.004,
 0.005,
 0.008,
 0.004,
 0.005,
 0.004,
 0.005,
 0.004,
 0.004,
 0.002,
 0.002,
 0.002,
 0.002,
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]

In [75]:
data['states']['Texas']

{'Alaska': [2568,
  283,
  1917,
  368,
  1070,
  89,
  839,
  142,
  1012,
  91,
  747,
  174,
  486,
  103,
  331,
  52,
  -1950,
  -695,
  -956,
  -299,
  -1038,
  -394,
  -509,
  -135,
  -669,
  -206,
  -332,
  -131,
  -243,
  -95,
  -115,
  -33,
  618,
  -412,
  961,
  69,
  32,
  -305,
  330,
  7,
  343,
  -115,
  415,
  43,
  243,
  8,
  216,
  19],
 'Alabama': [8398,
  4277,
  3146,
  975,
  3679,
  1857,
  1421,
  401,
  2537,
  1127,
  1014,
  396,
  2182,
  1293,
  711,
  178,
  -9683,
  -3184,
  -5159,
  -1340,
  -3390,
  -1107,
  -1803,
  -480,
  -3605,
  -985,
  -2015,
  -605,
  -2688,
  -1092,
  -1341,
  -255,
  -1285,
  1093,
  -2013,
  -365,
  289,
  750,
  -382,
  -79,
  -1068,
  142,
  -1001,
  -209,
  -506,
  201,
  -630,
  -77],
 'Arkansas': [13566,
  8781,
  3499,
  1286,
  5334,
  3318,
  1550,
  466,
  3921,
  2329,
  1106,
  486,
  4311,
  3134,
  843,
  334,
  -16572,
  -5780,
  -9038,
  -1754,
  -5059,
  -1673,
  -2822,
  -564,
  -5957,
  -1761,
  -3448,
  -7

In [58]:
data['states']['Alaska']['Texas'][0] 
+ data['states']['Alaska']['Texas'][16] + data['states']['Alaska']['Texas'][32]

-1236

In [59]:
data['states']['Alaska']['Texas'][0] 

1950

In [60]:
data['states']['Alaska']['Texas'][16] 

-2568

In [61]:
data['states']['Alaska']['Texas'][32] 

-618

In [30]:
len(data['states']['Hawaii']['Alaska'])

48

In [45]:
data['states']['Ohio'].keys()

dict_keys(['Alaska', 'Alabama', 'Arkansas', 'Arizona', 'California', 'Colorado', 'Connecticut', 'DistrictOfColumbia', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Iowa', 'Idaho', 'Illinois', 'Indiana', 'Kansas', 'Kentucky', 'Louisiana', 'Massachusetts', 'Maryland', 'Maine', 'Michigan', 'Minnesota', 'Missouri', 'Mississippi', 'Montana', 'NorthCarolina', 'NorthDakota', 'Nebraska', 'NewHampshire', 'NewJersey', 'NewMexico', 'Nevada', 'NewYork', 'Oklahoma', 'Oregon', 'Pennsylvania', 'RhodeIsland', 'SouthCarolina', 'SouthDakota', 'Tennessee', 'Texas', 'Utah', 'Virginia', 'Vermont', 'Washington', 'Wisconsin', 'WestVirginia', 'Wyoming', 'all_markets', 'states_all', 'largest_25', 'largest_10', 'top_25_list'])

In [10]:
flows = []
for source_state in data['states'].keys():
    print(source_state)
    state_data = data['states'][source_state]
    for dest_state in state_data.keys():
        if source_state == "Texas" or dest_state == "Texas":
            if dest_state not in [ 'states_all', 'largest_25', 'largest_10', 'top_25_list']:
                print(dest_state)
                print(state_data[dest_state])
                count = state_data[dest_state][0]
                flow = {
                    'origin': source_state,
                    'dest': dest_state,
                    'count': count
                }
                flows.append(flow)

flows

Alaska
Texas
[0.098, 0.085, 0.109, 0.102, 0.101, 0.091, 0.108, 0.106, 0.112, 0.093, 0.123, 0.121, 0.067, 0.057, 0.082, 0.058, 0.087, 0.066, 0.09, 0.09, 0.091, 0.072, 0.091, 0.106, 0.094, 0.075, 0.097, 0.094, 0.068, 0.056, 0.075, 0.057, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
Alabama
Texas
[0.067, 0.049, 0.084, 0.076, 0.068, 0.052, 0.08, 0.087, 0.069, 0.047, 0.086, 0.075, 0.063, 0.047, 0.087, 0.064, 0.077, 0.072, 0.081, 0.09, 0.087, 0.083, 0.09, 0.102, 0.074, 0.067, 0.077, 0.091, 0.067, 0.064, 0.071, 0.068, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

[{'origin': 'Alaska', 'dest': 'Texas', 'count': 0.098},
 {'origin': 'Alabama', 'dest': 'Texas', 'count': 0.067},
 {'origin': 'Arkansas', 'dest': 'Texas', 'count': 0.191},
 {'origin': 'Arizona', 'dest': 'Texas', 'count': 0.052},
 {'origin': 'California', 'dest': 'Texas', 'count': 0.075},
 {'origin': 'Colorado', 'dest': 'Texas', 'count': 0.118},
 {'origin': 'Connecticut', 'dest': 'Texas', 'count': 0.024},
 {'origin': 'DistrictOfColumbia', 'dest': 'Texas', 'count': 0.034},
 {'origin': 'Delaware', 'dest': 'Texas', 'count': 0.015},
 {'origin': 'Florida', 'dest': 'Texas', 'count': 0.041},
 {'origin': 'Georgia', 'dest': 'Texas', 'count': 0.052},
 {'origin': 'Hawaii', 'dest': 'Texas', 'count': 0.065},
 {'origin': 'Iowa', 'dest': 'Texas', 'count': 0.054},
 {'origin': 'Idaho', 'dest': 'Texas', 'count': 0.035},
 {'origin': 'Illinois', 'dest': 'Texas', 'count': 0.065},
 {'origin': 'Indiana', 'dest': 'Texas', 'count': 0.041},
 {'origin': 'Kansas', 'dest': 'Texas', 'count': 0.091},
 {'origin': 'Kent

In [11]:
flows

[{'origin': 'Alaska', 'dest': 'Texas', 'count': 0.098},
 {'origin': 'Alabama', 'dest': 'Texas', 'count': 0.067},
 {'origin': 'Arkansas', 'dest': 'Texas', 'count': 0.191},
 {'origin': 'Arizona', 'dest': 'Texas', 'count': 0.052},
 {'origin': 'California', 'dest': 'Texas', 'count': 0.075},
 {'origin': 'Colorado', 'dest': 'Texas', 'count': 0.118},
 {'origin': 'Connecticut', 'dest': 'Texas', 'count': 0.024},
 {'origin': 'DistrictOfColumbia', 'dest': 'Texas', 'count': 0.034},
 {'origin': 'Delaware', 'dest': 'Texas', 'count': 0.015},
 {'origin': 'Florida', 'dest': 'Texas', 'count': 0.041},
 {'origin': 'Georgia', 'dest': 'Texas', 'count': 0.052},
 {'origin': 'Hawaii', 'dest': 'Texas', 'count': 0.065},
 {'origin': 'Iowa', 'dest': 'Texas', 'count': 0.054},
 {'origin': 'Idaho', 'dest': 'Texas', 'count': 0.035},
 {'origin': 'Illinois', 'dest': 'Texas', 'count': 0.065},
 {'origin': 'Indiana', 'dest': 'Texas', 'count': 0.041},
 {'origin': 'Kansas', 'dest': 'Texas', 'count': 0.091},
 {'origin': 'Kent

In [12]:
write_data_json(flows,dest_data_path)

Wrote intermarket json data to  /Users/alanmccann/Dropbox/bain/bain-uploads/intermarketdata-20230111-flows.json


In [20]:
market_mapping_settings_path = '/Users/alanmccann/Dropbox/bain/bain-uploads/v5_12th_run Brian Stobie/Market_Mapping_Settings.csv'
dest_data_path = '/Users/alanmccann/Dropbox/bain/bain-uploads/intermarketdata-20230111-locations.json'

In [21]:
#     data_rows = []
#     with open(csv_file_path, encoding='utf-8-sig') as csvfile:
#         reader = csv.DictReader(csvfile)
#         for row in reader:
#             data_rows.append(row)

def read_market_settings(file_path):
    data_rows = []
    with open(file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    return  data_rows

def write_intermarket_locations_json(json_data, path):
    json_string = json.dumps(json_data)
    f = open(path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote intermarket locations json data to  {path}')


In [22]:
data = read_market_settings(market_mapping_settings_path)
data

[{'MarketName': 'AbileneTX',
  'Market_Display_Name': 'Abilene, TX',
  'StateFips': '',
  'CenterLat': '32.453549',
  'CenterLong': '-99.730981',
  'GeoTypeGroup': '1 - Designated Metro Areas',
  'DefaultZoomLevel': '8'},
 {'MarketName': 'AkronOH',
  'Market_Display_Name': 'Akron, OH',
  'StateFips': '',
  'CenterLat': '41.080491',
  'CenterLong': '-81.517454',
  'GeoTypeGroup': '1 - Designated Metro Areas',
  'DefaultZoomLevel': '8'},
 {'MarketName': 'AlbanyGA',
  'Market_Display_Name': 'Albany, GA',
  'StateFips': '',
  'CenterLat': '31.577248',
  'CenterLong': '-84.152334',
  'GeoTypeGroup': '1 - Designated Metro Areas',
  'DefaultZoomLevel': '8'},
 {'MarketName': 'AlbanyOR',
  'Market_Display_Name': 'Albany, OR',
  'StateFips': '',
  'CenterLat': '44.635272',
  'CenterLong': '-123.10634',
  'GeoTypeGroup': '1 - Designated Metro Areas',
  'DefaultZoomLevel': '8'},
 {'MarketName': 'AlbanyNY',
  'Market_Display_Name': 'Albany-Schenectady-Troy, NY',
  'StateFips': '',
  'CenterLat': '4

In [23]:
data[381]

{'MarketName': 'Alabama',
 'Market_Display_Name': 'Alabama',
 'StateFips': '',
 'CenterLat': '32.7794',
 'CenterLong': '-86.8287',
 'GeoTypeGroup': '2 - States',
 'DefaultZoomLevel': '5'}

In [24]:
locations = []
for location in data[381:]:
    location = {
        "id": location['MarketName'],
        "name":location['Market_Display_Name'] ,
        "lat": location['CenterLat'],
        "lon": location['CenterLong']
    }
    locations.append(location)
locations

[{'id': 'Alabama', 'name': 'Alabama', 'lat': '32.7794', 'lon': '-86.8287'},
 {'id': 'Alaska', 'name': 'Alaska', 'lat': '62.46751', 'lon': '-145.456457'},
 {'id': 'Arizona', 'name': 'Arizona', 'lat': '34.2744', 'lon': '-111.6602'},
 {'id': 'Arkansas', 'name': 'Arkansas', 'lat': '34.8938', 'lon': '-92.4426'},
 {'id': 'California',
  'name': 'California',
  'lat': '37.1841',
  'lon': '-119.4696'},
 {'id': 'Colorado', 'name': 'Colorado', 'lat': '38.9972', 'lon': '-105.5478'},
 {'id': 'Connecticut',
  'name': 'Connecticut',
  'lat': '41.6219',
  'lon': '-72.7273'},
 {'id': 'Delaware', 'name': 'Delaware', 'lat': '38.9896', 'lon': '-75.505'},
 {'id': 'DistrictOfColumbia',
  'name': 'DistrictOfColumbia',
  'lat': '38.9101',
  'lon': '-77.0147'},
 {'id': 'Florida', 'name': 'Florida', 'lat': '28.6305', 'lon': '-82.4497'},
 {'id': 'Georgia', 'name': 'Georgia', 'lat': '32.6415', 'lon': '-83.4426'},
 {'id': 'Hawaii', 'name': 'Hawaii', 'lat': '20.813579', 'lon': '-156.485423'},
 {'id': 'Idaho', 'nam

In [25]:
write_intermarket_locations_json(locations,dest_data_path)

Wrote intermarket locations json data to  /Users/alanmccann/Dropbox/bain/bain-uploads/intermarketdata-20230111-locations.json


In [25]:
locations

[{'id': 'Alabama', 'name': 'Alabama', 'lat': '32.7794', 'lon': '-86.8287'},
 {'id': 'Alaska', 'name': 'Alaska', 'lat': '62.46751', 'lon': '-145.456457'},
 {'id': 'Arizona', 'name': 'Arizona', 'lat': '34.2744', 'lon': '-111.6602'},
 {'id': 'Arkansas', 'name': 'Arkansas', 'lat': '34.8938', 'lon': '-92.4426'},
 {'id': 'California',
  'name': 'California',
  'lat': '37.1841',
  'lon': '-119.4696'},
 {'id': 'Colorado', 'name': 'Colorado', 'lat': '38.9972', 'lon': '-105.5478'},
 {'id': 'Connecticut',
  'name': 'Connecticut',
  'lat': '41.6219',
  'lon': '-72.7273'},
 {'id': 'Delaware', 'name': 'Delaware', 'lat': '38.9896', 'lon': '-75.505'},
 {'id': 'DistrictOfColumbia',
  'name': 'DistrictOfColumbia',
  'lat': '38.9101',
  'lon': '-77.0147'},
 {'id': 'Florida', 'name': 'Florida', 'lat': '28.6305', 'lon': '-82.4497'},
 {'id': 'Georgia', 'name': 'Georgia', 'lat': '32.6415', 'lon': '-83.4426'},
 {'id': 'Hawaii', 'name': 'Hawaii', 'lat': '20.813579', 'lon': '-156.485423'},
 {'id': 'Idaho', 'nam

In [26]:
loc_df = pd.DataFrame.from_dict(locations)
loc_df

,id,name,lat,lon
0,Alabama,Alabama,32.7794,-86.8287
1,Alaska,Alaska,62.46751,-145.456457
2,Arizona,Arizona,34.2744,-111.6602
3,Arkansas,Arkansas,34.8938,-92.4426
4,California,California,37.1841,-119.4696
5,Colorado,Colorado,38.9972,-105.5478
6,Connecticut,Connecticut,41.6219,-72.7273
7,Delaware,Delaware,38.9896,-75.505
8,DistrictOfColumbia,DistrictOfColumbia,38.9101,-77.0147
9,Florida,Florida,28.6305,-82.4497


In [28]:
loc_df.to_csv('/Users/alanmccann/Dropbox/bain/bain-uploads/intermarketdata-20221109-locations.csv', index=False)

In [29]:
flows

[{'origin': 'Alaska', 'dest': 'Texas', 'count': 1950},
 {'origin': 'Alabama', 'dest': 'Texas', 'count': 9683},
 {'origin': 'Arkansas', 'dest': 'Texas', 'count': 16572},
 {'origin': 'Arizona', 'dest': 'Texas', 'count': 17752},
 {'origin': 'California', 'dest': 'Texas', 'count': 32136},
 {'origin': 'Colorado', 'dest': 'Texas', 'count': 31664},
 {'origin': 'Connecticut', 'dest': 'Texas', 'count': 2413},
 {'origin': 'DistrictOfColumbia', 'dest': 'Texas', 'count': 1401},
 {'origin': 'Delaware', 'dest': 'Texas', 'count': 948},
 {'origin': 'Florida', 'dest': 'Texas', 'count': 43255},
 {'origin': 'Georgia', 'dest': 'Texas', 'count': 18703},
 {'origin': 'Hawaii', 'dest': 'Texas', 'count': 2263},
 {'origin': 'Iowa', 'dest': 'Texas', 'count': 3929},
 {'origin': 'Idaho', 'dest': 'Texas', 'count': 3625},
 {'origin': 'Illinois', 'dest': 'Texas', 'count': 14236},
 {'origin': 'Indiana', 'dest': 'Texas', 'count': 7595},
 {'origin': 'Kansas', 'dest': 'Texas', 'count': 7958},
 {'origin': 'Kentucky', 'des

In [32]:
flows_df = pd.DataFrame.from_dict(flows)
flows_df

,origin,dest,count
0,Alaska,Texas,1950
1,Alabama,Texas,9683
2,Arkansas,Texas,16572
3,Arizona,Texas,17752
4,California,Texas,32136
...,...,...,...
96,Vermont,Texas,745
97,Washington,Texas,14965
98,Wisconsin,Texas,5953
99,WestVirginia,Texas,1310


In [33]:
flows_df.to_csv('/Users/alanmccann/Dropbox/bain/bain-uploads/intermarketdata-20221109-flows.csv', index=False)

In [19]:
flows

[{'origin': 'Alaska', 'dest': 'Texas', 'count': 0.098},
 {'origin': 'Alabama', 'dest': 'Texas', 'count': 0.067},
 {'origin': 'Arkansas', 'dest': 'Texas', 'count': 0.191},
 {'origin': 'Arizona', 'dest': 'Texas', 'count': 0.052},
 {'origin': 'California', 'dest': 'Texas', 'count': 0.075},
 {'origin': 'Colorado', 'dest': 'Texas', 'count': 0.118},
 {'origin': 'Connecticut', 'dest': 'Texas', 'count': 0.024},
 {'origin': 'DistrictOfColumbia', 'dest': 'Texas', 'count': 0.034},
 {'origin': 'Delaware', 'dest': 'Texas', 'count': 0.015},
 {'origin': 'Florida', 'dest': 'Texas', 'count': 0.041},
 {'origin': 'Georgia', 'dest': 'Texas', 'count': 0.052},
 {'origin': 'Hawaii', 'dest': 'Texas', 'count': 0.065},
 {'origin': 'Iowa', 'dest': 'Texas', 'count': 0.054},
 {'origin': 'Idaho', 'dest': 'Texas', 'count': 0.035},
 {'origin': 'Illinois', 'dest': 'Texas', 'count': 0.065},
 {'origin': 'Indiana', 'dest': 'Texas', 'count': 0.041},
 {'origin': 'Kansas', 'dest': 'Texas', 'count': 0.091},
 {'origin': 'Kent